In [1]:
#unzip downloaded files

#install library
!pip install zipfile36

#import library
import zipfile

#unzipping
train_data = zipfile.ZipFile('train.csv.zip')
test_data = zipfile.ZipFile('test.csv.zip')

#extract both data
train_data.extractall()
test_data.extractall()

#close both data
train_data.close()
test_data.close()

In [2]:
#import all necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
#load datasets
train = pd.read_csv('train.csv')
#test = pd.read_csv('test.csv')

In [4]:
train.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37670293 entries, 0 to 37670292
Data columns (total 24 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   date_time                  object 
 1   site_name                  int64  
 2   posa_continent             int64  
 3   user_location_country      int64  
 4   user_location_region       int64  
 5   user_location_city         int64  
 6   orig_destination_distance  float64
 7   user_id                    int64  
 8   is_mobile                  int64  
 9   is_package                 int64  
 10  channel                    int64  
 11  srch_ci                    object 
 12  srch_co                    object 
 13  srch_adults_cnt            int64  
 14  srch_children_cnt          int64  
 15  srch_rm_cnt                int64  
 16  srch_destination_id        int64  
 17  srch_destination_type_id   int64  
 18  is_booking                 int64  
 19  cnt                        int64  
 20  

Most of columns are numeric, so we can't do a lot of feature engineering. For example, user_location_location doesn't tell us what country represents which number. So we can't transform the data into features that better represent the predictor variables


#### Generating new features

New features are generated date_time, srch_ci (check-in date), and srch_co (check-out date)
- Year of access
- Month of access
- Day of Week of access
- Length of stey

In [6]:
train['date_time'] = pd.to_datetime(train['date_time'])
train['year'] = train['date_time'].dt.year
train['month'] = train['date_time'].dt.month
train['dayofweek'] = train['date_time'].dt.dayofweek
train["srch_ci"] = pd.to_datetime(train["srch_ci"], format='%Y-%m-%d', errors="coerce")
train["srch_co"] = pd.to_datetime(train["srch_co"], format='%Y-%m-%d', errors="coerce")
train["stay_span"] = (train["srch_co"] - train["srch_ci"]).astype('timedelta64[h]')
train.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,year,month,dayofweek,stay_span
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,3,2,50,628,1,2014,8,0,96.0
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,1,1,2,50,628,1,2014,8,0,96.0
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,2,50,628,1,2014,8,0,96.0
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,2,50,1457,80,2014,8,5,120.0
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,2,50,1457,21,2014,8,5,120.0


In [7]:
print(train.shape)
#print(test.shape)

(37670293, 28)


There are nearly 37.6 million rows. We need to make the dataset smaller as we do not have enough compute

In [8]:
print(train['user_id'].nunique())

1198786


These 37.6 million rows represent 1.2 million users. We will downsample the dataset to 10000 randomly chosen users

In [9]:
userids = list(train['user_id'].unique())  #gets list of unique user_ids

In [10]:
import random
random.seed(10)
#choose 10k random users
userids_10k = random.choices(userids, k=10000)

In [11]:
#create train dataset with the 10k users
train_10k = train[train['user_id'].isin(userids_10k)] 

In [12]:
train_10k.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,year,month,dayofweek,stay_span
1927,2014-02-27 07:24:03,2,3,66,174,23793,1780.4772,8654,1,0,...,0,2,2,50,637,47,2014,2,3,48.0
1928,2014-08-30 16:10:42,2,3,66,174,13277,133.1612,8654,1,0,...,0,1,2,50,970,80,2014,8,5,48.0
1929,2014-08-30 16:25:16,2,3,66,174,13277,124.8268,8654,1,0,...,0,1,2,50,368,10,2014,8,5,24.0
1930,2014-08-30 16:26:45,2,3,66,174,13277,112.2581,8654,1,0,...,0,1,2,50,368,21,2014,8,5,24.0
1931,2014-08-30 16:29:57,2,3,66,174,13277,117.6094,8654,1,0,...,0,3,2,50,368,95,2014,8,5,24.0


In [13]:
print(train_10k.shape)

(316422, 28)


This has reduced the training data set to 10000 users consisting of 316K rows

Now we want to remove the travel agents from the dataset so that it does not affect the single user preferences. We identify a travel agent as someone with more than 20 bookings.

In [14]:
#remove all non-bookings to make counting easier
train_10k_booking = train_10k[train_10k.is_booking != 0]
for user in userids_10k:
  #count the number of rows under a single user
  bookings = len(train_10k_booking.loc[train_10k_booking['user_id'] == user])
  if bookings >= 20:
    #remove the travel agent from dataset
    train_10k = train_10k[train_10k.user_id != user]

In [15]:
print(train_10k.shape)

(290825, 28)


#### Handling missing data

In [16]:
# Calculate total missing values per column
total_missing = train_10k.isnull().sum()

# Calculate % missing values per column
percent_missing = ((train_10k.isnull().sum()/train_10k.isnull().count())*100)

# Create a dataframe with the total and % missing values per column 
missing_data = pd.concat([total_missing, percent_missing], axis=1, keys=['Total Missing', 'Percent Missing'])
missing_data

,Total Missing,Percent Missing
date_time,0,0.000000
site_name,0,0.000000
posa_continent,0,0.000000
user_location_country,0,0.000000
user_location_region,0,0.000000
user_location_city,0,0.000000
orig_destination_distance,102986,35.411674
user_id,0,0.000000
is_mobile,0,0.000000
is_package,0,0.000000


In [17]:
#find mean orig_destination_distance
print(train_10k['orig_destination_distance'].mean())

1972.884946094794


In [18]:
#fill missing values for orig_destination_distance with the column mean
train_10k['orig_destination_distance'] = train_10k['orig_destination_distance'].fillna(1972.884946094794)

In [19]:
#clean data with empty cells
train_10k = train_10k.dropna()

In [20]:
print(train_10k.shape)

(290413, 28)


In [21]:
train_10k.to_csv('train_downsampled_10k.csv', index = False)

In [22]:
train_10k.corr()['hotel_cluster']

C:\Users\pragn\AppData\Local\Temp\ipykernel_20696\3626639521.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_10k.corr()['hotel_cluster']


site_name                   -0.024835
posa_continent               0.019831
user_location_country       -0.015561
user_location_region         0.013036
user_location_city          -0.001013
orig_destination_distance    0.011047
user_id                      0.001739
is_mobile                    0.009824
is_package                   0.042279
channel                     -0.003721
srch_adults_cnt              0.010422
srch_children_cnt            0.011673
srch_rm_cnt                 -0.007727
srch_destination_id         -0.016216
srch_destination_type_id    -0.036056
is_booking                  -0.023270
cnt                          0.002919
hotel_continent             -0.012584
hotel_country               -0.021196
hotel_market                 0.034559
hotel_cluster                1.000000
year                        -0.000776
month                        0.001328
dayofweek                    0.001842
stay_span                    0.039513
Name: hotel_cluster, dtype: float64

Since none of the columns are correlated with hotel_cluster, linear regression won't work well on our data.


In [23]:
train_10k.groupby(['year'])['month'].value_counts()

year  month
2013  7         8464
      3         7724
      6         7553
      8         7310
      5         7308
      9         7248
      4         7031
      12        6693
      10        6679
      2         6259
      11        6208
      1         5780
2014  12       23371
      7        22166
      8        21950
      11       21348
      9        20831
      10       20362
      6        17892
      5        13963
      4        13395
      3        13081
      1         9089
      2         8708
Name: month, dtype: int64

#### Creating datasets with more training data - in batches of 1k users

In [24]:
#choose 11k random users using same seed
userids_11k = random.choices(userids, k=11000)
#create train dataset with the 11k users
train_11k = train[train['user_id'].isin(userids_11k)]
print(train_11k.shape)
#remove all non-bookings to make counting easier
train_11k_booking = train_11k[train_11k.is_booking != 0]
for user in userids_11k:
  #count the number of rows under a single user
  bookings = len(train_11k_booking.loc[train_11k_booking['user_id'] == user])
  if bookings >= 20:
    #remove the travel agent from dataset
    train_11k = train_11k[train_11k.user_id != user]
print(train_11k.shape)

(348150, 28)
(322270, 28)


In [25]:
#find mean orig_destination_distance
print(train_11k['orig_destination_distance'].mean())

1987.4836235623675


In [26]:
#fill missing values for orig_destination_distance with the column mean
train_11k['orig_destination_distance'] = train_11k['orig_destination_distance'].fillna(1987.4836235623675)
#clean data with empty cells
train_11k = train_11k.dropna()
print(train_11k.shape)
train_11k.to_csv('train_downsampled_11k.csv', index = False)

(321845, 28)


In [27]:
#choose 12k random users using same seed
userids_12k = random.choices(userids, k=12000)
#create train dataset with the 12k users
train_12k = train[train['user_id'].isin(userids_12k)]
print(train_12k.shape)
#remove all non-bookings to make counting easier
train_12k_booking = train_12k[train_12k.is_booking != 0]
for user in userids_12k:
  #count the number of rows under a single user
  bookings = len(train_12k_booking.loc[train_12k_booking['user_id'] == user])
  if bookings >= 20:
    #remove the travel agent from dataset
    train_12k = train_12k[train_12k.user_id != user]
print(train_12k.shape)

(372664, 28)
(345106, 28)


In [28]:
#find mean orig_destination_distance
print(train_12k['orig_destination_distance'].mean())

1937.888871297059


In [29]:
#fill missing values for orig_destination_distance with the column mean
train_12k['orig_destination_distance'] = train_12k['orig_destination_distance'].fillna(1937.888871297059)
#clean data with empty cells
train_12k = train_12k.dropna()
print(train_12k.shape)
train_12k.to_csv('train_downsampled_12k.csv', index = False)

(344670, 28)


In [30]:
#choose 13k random users using same seed
userids_13k = random.choices(userids, k=13000)
#create train dataset with the 13k users
train_13k = train[train['user_id'].isin(userids_13k)]
print(train_13k.shape)
#remove all non-bookings to make counting easier
train_13k_booking = train_13k[train_13k.is_booking != 0]
for user in userids_13k:
  #count the number of rows under a single user
  bookings = len(train_13k_booking.loc[train_13k_booking['user_id'] == user])
  if bookings >= 20:
    #remove the travel agent from dataset
    train_13k = train_13k[train_13k.user_id != user]
print(train_13k.shape)

(401915, 28)
(373407, 28)


In [31]:
#find mean orig_destination_distance
print(train_13k['orig_destination_distance'].mean())

1994.0424972621383


In [32]:
#fill missing values for orig_destination_distance with the column mean
train_13k['orig_destination_distance'] = train_13k['orig_destination_distance'].fillna(1994.0424972621383)
#clean data with empty cells
train_13k = train_13k.dropna()
print(train_13k.shape)
train_13k.to_csv('train_downsampled_13k.csv', index = False)

(372899, 28)


In [33]:
#choose 14k random users using same seed
userids_14k = random.choices(userids, k=14000)
#create train dataset with the 14k users
train_14k = train[train['user_id'].isin(userids_14k)]
print(train_14k.shape)
#remove all non-bookings to make counting easier
train_14k_booking = train_14k[train_14k.is_booking != 0]
for user in userids_14k:
  #count the number of rows under a single user
  bookings = len(train_14k_booking.loc[train_14k_booking['user_id'] == user])
  if bookings >= 20:
    #remove the travel agent from dataset
    train_14k = train_14k[train_14k.user_id != user]
print(train_14k.shape)

(442084, 28)
(406843, 28)


In [34]:
#find mean orig_destination_distance
print(train_14k['orig_destination_distance'].mean())

1945.0912953497989


In [35]:
#fill missing values for orig_destination_distance with the column mean
train_14k['orig_destination_distance'] = train_14k['orig_destination_distance'].fillna(1945.0912953497989)
#clean data with empty cells
train_14k = train_14k.dropna()
print(train_14k.shape)
train_14k.to_csv('train_downsampled_14k.csv', index = False)

(406249, 28)


In [36]:
#choose 15k random users using same seed
userids_15k = random.choices(userids, k=15000)
#create train dataset with the 15k users
train_15k = train[train['user_id'].isin(userids_15k)]
print(train_15k.shape)
#remove all non-bookings to make counting easier
train_15k_booking = train_15k[train_15k.is_booking != 0]
for user in userids_15k:
  #count the number of rows under a single user
  bookings = len(train_15k_booking.loc[train_15k_booking['user_id'] == user])
  if bookings >= 20:
    #remove the travel agent from dataset
    train_15k = train_15k[train_15k.user_id != user]
print(train_15k.shape)

(478268, 28)
(436883, 28)


In [37]:
#find mean orig_destination_distance
print(train_15k['orig_destination_distance'].mean())

1955.5055139469346


In [38]:
#fill missing values for orig_destination_distance with the column mean
train_15k['orig_destination_distance'] = train_15k['orig_destination_distance'].fillna(1955.5055139469346)
#clean data with empty cells
train_15k = train_15k.dropna()
print(train_15k.shape)
train_15k.to_csv('train_downsampled_15k.csv', index = False)

(436347, 28)
